Use this version of the notebook if you want to run everything interactively. 

*To run cells with code, `shift + enter`* 

*To restart the session, `Kernel -> Restart and clear output`*

# Dihedral Kullback-Leibler (KL) divergence
----

----
## Introduction 

-------

### Some details on trajectory/topology input

-------

#### Simulation input

To run these scripts, you should have two (or more) different trajectories for the same protein, with different effectors bound. In these examples, the simulations have different allosteric ligands. 



#### Topology

It is important that the data is first processed to ensure the same residues are present. This script will calculate psi/phi/chi in the order which they are in the topology - so in order for the script to work you need to have identical order of residues between the different simulations. 



#### Trajectory 

All of these scripts will take input trajectories which are accepted by **[mdtraj](http://mdtraj.org/1.9.0/load_functions.html "mdtraj.Trajectory")**.

There are some basic scripts in the folder **`Scripts/Traj_processing_scripts`** for processing trajectory and topology using parmed or cpptraj. Remove any protein residues not common to both trajectory and also you can remove water as it will not be needed. 

As long as you have the same sequence of residues for the protein in each case, there is no need to remove any substrate,  ligand or ions, but it is useful to do also this now if you are also going to carry out a PCA. 



####  KL

Usually for each KL calculation, two different simulations are compared. These could be a simulation with no ligand bound and one with a ligand. Or two simulations with different ligands (i.e. activating or inhibiting ligands). 

Also in the scripts folder is a script to create two smaller trajectory from one larger one. It is useful to calculate the KL divergence of one system relative to itself, using two different (or one separated) trajectories, in order to take account of noise. 

----


### Kullback-Leibler Divergence 

---
**KL divergence is defined as:**

\begin{equation}
D_{KL} (P\|Q)=\sum _{i}P(i)\,\log {\frac {P(i)}{Q(i)}}
\end{equation}

In this case, distribution P will be the data obtained from one simulation *(i.e. protein with activator bound)* and distribution Q will be that obtained from another simulation *(i.e. protein with inhibitor bound)*.

**P** and **Q** refer to distributions of a particular descriptor, for example a particular Psi angle, and we can collect the value of that Psi angle as a distribution over the trajectory. 

So for the activator bound simulation we could obtain the distibution in green for **P**, and for the inhibitor bound simulation we obtain the distribution in blue for **Q**. 

* In the first case, the distributions are similar, and so the mean value will be similar, and the KL divergence will be small. 


* In the second case, the mean value is similar, but the KL divergence will be higher since the distributions are not the same across all bins. 


* In the third case, the mean value is different and the KL divergence is higher. 



<img src="z_images/distributions.png">

-----------


### Overall workflow

-----------

So to summarise the overall workflow:

1. Run simulations. 

2. Calculate distributions of torsions. 

3. Compute KL between the different systems. 

4. Input KL as a B-factor into the pdb to visualise.

<img src="z_images/KL_calculation.png" width="800" >

---
##### Some notes on the workflow: 

* Step 1 of the workflow: Run simulations and process the trajectories to ensure you have identical topology / order of residues.


* Step 2 of the workflow: The first script calculates distributions of Psi, Phi, Chi1 and Chi2 from a trajectory. These distributions are output as a text file for each torsion, into folders PSI, PHI, CHI1, CHI2 in the directory OUTPUT. In step 2 we also have to assign each torsion to the correct residue.


* Step 3 of the workflow: KL values are then calculated with another script which just loops through the files output by the first script, and these KL values can then be input onto the structure to highlight the high KL regions, for Psi, Phi, Chi1 and Chi2 separately; or grouped as either backbone or sidechain. 


* Step 4 of the workflow: A pymol script will load the output from the KL calculation to be visualised on the structure.


# <span style="color:teal"> Step 1 of the workflow: </span>
## <span style="color:teal"> Run simulations and set up folders to run the analysis </span>

-------

--------

### 1.1 Folder and filenames

-------

#### *<span style="color:red">The folders for trajectories are already set up for the tutorial: so there is no need to set up these folders for the tutorial; below for info only.</span> *


To run on other systems set up the following folder structure to complete the analysis.

**`mkdir 0_Analysis`**  

Then set up a folder for trajectory data.

**`cd 0_Analysis
mkdir 0_TRAJECTORIES`**

Within 0_TRAJECTORIES make a folder for each system, and inside these folders either put the trajectory and topology files, or soft links to them. Also include a pdb file with the same topology as the trajectory (usually just take the first frame of the simulation).

**`cd 0_TRAJECTORIES
mkdir 0_system_A
mkdir 1_system_B`**

**`cd 0_system_A
ln -s /home/somefilelocation/traj0001.dcd
ln -s /home/somefilelocation/topology.parm7
ln -s /home/somefilelocation/first_frame.pdb`**

Within each you can have one or more trajectories. For example, if you have a long trajectory which was run in segments, or if you want to use many different runs of the same system in order to use more snapshots, just input these separately in "filename_list":

*e.g.* 
*filename_list_system_A = ["traj1.dcd" , "traj2.dcd" , "traj3.dcd" , "traj4.dcd" , "traj5.dcd"]*

<span style="color:red"> *(We plan to change this in order just to automatically pull all files with a particular extention (e.g. .dcd) into these lists: but for now they have to be input)*</span>


### 1.2 Simulations for this tutorial
------


For this example, two systems are used: 

**0_system_A**  (*PDK1 with allosteric inhibitor 1F8 bound. PDB ID [3ORX](https://www.rcsb.org/structure/3ORX)*)

**1_system_B**  (*PDK1 with allosteric activator 2A2 bound. PDB ID [3ORZ](https://www.rcsb.org/structure/3orz).*)

And within each of these folders are two trajectory files (in .dcd format) along with the topology file (in .parm7 format). 

traj0001.dcd and traj0002.dcd are both 500 ns sections of the same simulation which have been post processed to remove everything other than the protein (water, ligand, ATP, Mg2+ ions and substrate peptide are present in the original simulation but removed for analysis).

Ideally have a tracectory with many frames (save a snapshot around every 10 ps or so). 

The trajectories included here initially have 100k snapshots for 500 ns simulation, but have been reduced to 2000 snapshots per 500 ns for the purpose of the tutorial. 

-----

Check we are in the correct folder `0_Analysis`:

In [1]:
pwd

u'/home/t702348/lisa/X_PDK1_tutorial/0_Analysis'

Check that filenames within each folder are correctly set up

In [2]:
ls 0_TRAJECTORIES/0_system_A/

cap_traj/        fulltraj.dcd              short_traj_aligned.dcd  traj0002.dcd
cpptraj.log      longtraj_aligned_PCA.dcd  topology.parm7
first_frame.pdb  parmed.log                traj0001.dcd


In [3]:
ls 0_TRAJECTORIES/1_system_B/

cap_traj/             fulltraj.dcd              short_traj_aligned.dcd
cpptraj.log           longtraj_aligned_PCA.dcd  topology.parm7
first_frame.pdb       parmed.log                traj0001.dcd
first_frame_test.pdb  Saved_first_frame.pdb     traj0002.dcd



---

---

---

# <span style="color:teal"> Step 2 of the workflow: </span>
## <span style="color:teal"> Calculating dihedrals with 1_Dihedral.py </span>

----

----

The following packages are required: 

In [4]:
import scipy as sp
import numpy as np
import mdtraj as md
import sys
import os
import math
get_ipython().magic(u'pylab inline')

Populating the interactive namespace from numpy and matplotlib


We will run the script for one system at a time  and restart the script between runs. In this case we are only running for 2 systems **0_system_A** and **1_system_B**.

To do this, just change the value of `input_system` to either 0 or 1, which will run the script for 0_system_A or 1_system_B respectively. 

Usually if many systems should be run, this script can be run via a bash script (`Scripts/1_dihedral_many_systems.sh`) which will loop through this using different values for `input_system`.

### 2.1. Selecting input and setting trajectory/topology filenames
----

Select the system to run using `input_system`. First run as `input_system = 0` and then later as `input_system = 1`

In [5]:
input_system = 0

In [6]:
# system_list with folder names of different systems. 
system_list = ["0_system_A","1_system_B"]

# Input one or more trajectory names into filname list. 
filename_list = ["traj0001.dcd","traj0002.dcd"]

topology_filename = "topology.parm7"

md_data = ["0_TRAJECTORIES"]

### 2.2. Make lists of directories to trajectory data 
---

Based on the info given above, the below 2 cells just create lists of file locations for trajectories. 

In [7]:
# Make a list with all file locations of trajectory data
all_files_list = []

for i in range(0,len(system_list)):
    for j in range(0,len(filename_list)):
        filenames = "%s/%s/%s" % (md_data[0],system_list[i],filename_list[j])
        all_files_list.append(filenames)

In [8]:
# Make a list of lists to separate file locations for each simulation.
input_files = []
for i in range(0,len(system_list)):
    inside_list = []
    for j in range(0,len(filename_list)):
        filenames = "%s/%s/%s" % (md_data[0],system_list[i],filename_list[j])
        inside_list.append(filenames)
    input_files.append(inside_list)
print (input_files)

[['0_TRAJECTORIES/0_system_A/traj0001.dcd', '0_TRAJECTORIES/0_system_A/traj0002.dcd'], ['0_TRAJECTORIES/1_system_B/traj0001.dcd', '0_TRAJECTORIES/1_system_B/traj0002.dcd']]




### 2.3. Create folders for the output files
---

In [9]:
for i in system_list:
    if not os.path.exists("1_DIHEDRALS/%s/OUTPUT/PSI" % i):
        filename = "1_DIHEDRALS/%s/OUTPUT/PSI" % i
        cmd = "mkdir -p %s" % filename
        os.system(cmd)
        
for i in system_list:
    if not os.path.exists("1_DIHEDRALS/%s/OUTPUT/PHI" % i):
        filename = "1_DIHEDRALS/%s/OUTPUT/PHI" % i
        cmd = "mkdir -p %s" % filename
        os.system(cmd)
        
for i in system_list:
    if not os.path.exists("1_DIHEDRALS/%s/OUTPUT/CHI1" % i):
        filename = "1_DIHEDRALS/%s/OUTPUT/CHI1" % i
        cmd = "mkdir -p %s" % filename
        os.system(cmd)
        
for i in system_list:
    if not os.path.exists("1_DIHEDRALS/%s/OUTPUT/CHI2" % i):
        filename = "1_DIHEDRALS/%s/OUTPUT/CHI2" % i
        cmd = "mkdir -p %s" % filename
        os.system(cmd)  

In [10]:
# Check the system we have selected with "input_system".

input_files[input_system]

['0_TRAJECTORIES/0_system_A/traj0001.dcd',
 '0_TRAJECTORIES/0_system_A/traj0002.dcd']

### 2.4. Load the trajectory
---

In [11]:
# Start by loading the first trajectory

traj = md.load_dcd(input_files[input_system][0] , top="%s/%s/%s" % (md_data[0],system_list[input_system],topology_filename))
top = traj.topology
print ("One trajectory file traj0001.dcd is: ") 
print (traj)

# Then loop through the remaining trajectories to add them to the first.

for i in range(1,len(filename_list)):
    traj_ADD = md.load_dcd(input_files[input_system][i] , top="%s/%s/%s" % (md_data[0],system_list[input_system],topology_filename))
    traj = traj.join(traj_ADD,check_topology=True, discard_overlapping_frames=False)
    
print ("Combined traj0001.dcd and traj0002.dcd is: ") 
print (traj)

One trajectory file traj0001.dcd is: 
<mdtraj.Trajectory with 2000 frames, 4676 atoms, 285 residues, and unitcells>
Combined traj0001.dcd and traj0002.dcd is: 
<mdtraj.Trajectory with 4000 frames, 4676 atoms, 285 residues, and unitcells>


#### *Here you can save the first frame as a pdb*

For the tutorial, in order to load the results in pymol, there is already a pdb for each in the 0_TRAJECTORIES folder. 

If you don't already have a pdb, you can use the below to save for each system.

`traj[0].save_pdb("%s/%s/first_frame.pdb" % (md_data[0],system_list[input_system]))`

### 2.5. Compute dihedrals
---

We use mdtraj to compute the dihedrals. The output of this: 

**psi_list[0]** gives the *atom indices* making up each psi torsion.

**psi_list[1]** gives the *value of psi* for each residue, for *each snapshot*.


#### Torsions calculated 

We have only included Psi and Phi for the backbone, and Chi1 and Chi2 for the sidechain torsions at this stage. However **mdtraj** can compute Psi, Phi, Omega, Chi1, Chi2, Chi3 and Chi4 so this could be extended. 


#### Memory issues:
We loaded the whole trajectory to memory with `md.load_dcd` by loading traj0001.dcd and traj0002.dcd and joining them together; however this may not always be possible if you have limited memory, or a very large trajectory. 

You can also use **[md.iterload](http://mdtraj.org/1.9.0/api/generated/mdtraj.iterload.html "mdtraj.iterload")** instead of initally loading the entire trajectory to memory. This would be done as follows:
 
#### Make lists to collect torsions: 

`psi_list = []
phi_list = []
chi1_list = []
chi2_list = []`

#### Use `md.iterload` to calculate torsions and append them to each list:

`for chunk in md.iterload(trajfile, chunk=100, top=top):
	psi_list.append(md.compute_psi(chunk,periodic=True,opt=True))
	phi_list.append(md.compute_phi(chunk,periodic=True,opt=True))
	chi1_list.append(md.compute_chi1(chunk,periodic=True,opt=True))
	chi2_list.append(md.compute_chi2(chunk,periodic=True,opt=True))`    
    

#### With `iterload`, the output is slightly different to `md.load`

Further scripts will be set up to run this with iterload. 

In [12]:
psi_list = md.compute_psi(traj, periodic=True, opt=True)
phi_list = md.compute_phi(traj, periodic=True, opt=True)
chi1_list = md.compute_chi1(traj, periodic=True, opt=True)
chi2_list = md.compute_chi2(traj, periodic=True, opt=True)

number_angles_list = []
number_angles_list.append(len(psi_list[0][:,0]))
number_angles_list.append(len(phi_list[0][:,0]))
number_angles_list.append(len(chi1_list[0][:,0]))
number_angles_list.append(len(chi2_list[0][:,0]))
number_angles_array = np.vstack((np.array(['Number of psi','Number of phi','Number of chi1','Number of chi2']),np.array(number_angles_list))).T

print (" We have the following numbers of psi, phi, chi1 and chi2 for this system: ") 
print (number_angles_array)

np.savetxt("1_DIHEDRALS/number_angles.dat",number_angles_array,fmt='%s')

 We have the following numbers of psi, phi, chi1 and chi2 for this system: 
[['Number of psi' '284']
 ['Number of phi' '284']
 ['Number of chi1' '252']
 ['Number of chi2' '204']]


##### Some notes on format of the output of md.compute_psi

`psi_list[0]` is the atom indices making up each psi.

In [13]:
print (psi_list[0])

[[   0    2   22   24]
 [  24   26   44   46]
 [  46   48   66   68]
 ...
 [4607 4617 4619 4621]
 [4621 4623 4641 4643]
 [4643 4645 4660 4662]]


Everything is zero indexed, so for the first residue, this will be `psi_list[0][0]`

In [14]:
print psi_list[0][0]

[ 0  2 22 24]


`psi_list[1]` is the value of each psi, for each snapshot. So should be an array of `number_of_snapshots` x `number_of_psi` 

In [15]:
print (psi_list[1]).shape

(4000, 284)


To look at one particular psi, `psi_list[1][:,i]` is the value of each psi(i) for each frame of traj. 

The number i is zero indexed, so `psi_list[1][:,0]` is the first psi angle, for every snapshot.

In [16]:
print (psi_list[1][:,0])

[2.1758876 1.8641511 2.698718  ... 2.8297236 3.015326  2.6422095]



### 2.6. Outputting a file with CA indexes for each psi, phi, chi1 and chi2

---


The rest of this script saves each torsional angle numbered Psi(0) to Psi(n), Chi1(0) to Chi1(m), etc, and not per residue: so we need to match up which atom indexes match to which Psi, Phi, Chi1 & Chi2. 

The next cell saves a file with: 

    
| CA index | PSI | PHI | CHI1 | CHI2 |
| :--- | :-: | :-: | :--: | :--: |
|7 |0 |0 |0| none|
|18| 1 |1 |1| 0|
|30| 2 |2| 2 |1|
|52 |3| 3 |3| 2|
|66| 4 |4 |4 |none|
|82 |5 |5 |none |none|
|92 |6 |6| 5| 3|




Where the first column are the C$\alpha$ indexes and then which Psi Phi Chi1 Chi2 should be assigned to that C$\alpha$. Not all cases have Chi1 and Chi2 and so for those C$\alpha$ they will show "none". 

We will run the script for each system, saving this file to the folder associated with that system in order to later check that the atom indices that we are calculating are the same for each torsion computed. 

After you have run the below cell for both systems, you can use vimdiff on the file output for each case: 

`$ vimdiff 1_DIHEDRALS/0_system_A/DIHEDRALS_CA_angles_indexes.dat 1_DIHEDRALS/1_system_B/DIHEDRALS_CA_angles_indexes.dat`

In [17]:
c_alphas = top.select("name CA")
c_alphas_list = []
for i in c_alphas:
    c_alphas_list.append(i)

psi_ca_list = []
for i in psi_list[0][:,1]:
    psi_ca_list.append(i)
phi_ca_list = []
for i in phi_list[0][:,2]:
    phi_ca_list.append(i)
chi1_ca_list = []
for i in chi1_list[0][:,1]:
    chi1_ca_list.append(i)
chi2_ca_list = []
for i in chi2_list[0][:,0]:
    chi2_ca_list.append(i)

psi_all_list = []
phi_all_list = []
chi1_all_list = []
chi2_all_list = []

for i in xrange(0, len(c_alphas)):
    CA_index = c_alphas[i]
    if CA_index in psi_list[0][:,1]:
        psi_all_list.append(psi_ca_list.index(CA_index))
    else:
        psi_all_list.append("none")
    if CA_index in phi_list[0][:,2]:
        phi_all_list.append(phi_ca_list.index(CA_index))
    else:
        phi_all_list.append("none")
    if CA_index in chi1_list[0][:,1]:
        chi1_all_list.append(chi1_ca_list.index(CA_index))
    else:
        chi1_all_list.append("none")
    if CA_index in chi2_list[0][:,0]:
        chi2_all_list.append(chi2_ca_list.index(CA_index))
    else:
        chi2_all_list.append("none")

psi_all_arr = np.array(psi_all_list)
phi_all_arr = np.array(phi_all_list)
chi1_all_arr = np.array(chi1_all_list)
chi2_all_arr = np.array(chi2_all_list)

CA_psi_phi_chi1_chi2_indexes = np.vstack((c_alphas,psi_all_arr,phi_all_arr,chi1_all_arr,chi2_all_arr))
CA_psi_phi_chi1_chi2_indexes = CA_psi_phi_chi1_chi2_indexes.T
np.savetxt("1_DIHEDRALS/%s/DIHEDRALS_CA_angles_indexes.dat" % system_list[input_system],CA_psi_phi_chi1_chi2_indexes,fmt='%s')

### 2.7. Computing distributions of dihedrals
---

From the md.compute_psi, we have an list which is *DIHEDRAL* vs *SNAPSHOT*. We need to calculate for every dihedral a probability distribution. 

Also when calculating KL, **both distributions should be continuous**. That is, if P(i)=0 or Q(i)=0 then there is no solution. In order to avoid this problem, a negligibly small value is added to any zero-count bins. 

#### Number of bins 

The number of bins will depend on the number of snapshots used. 

For a simulation of 1 $\mu$s, usually we save around 200k snapshots. Around 300 bins is appropriate for this number of datapoints. In this example, we have 4000 snapshots and so 60 bins are used.

If using the script `Scripts/1_dihedral.py`, the number of bins is passed as an argument.

In [18]:
number_of_bins = 60

### PSI

In [19]:
for i in xrange(psi_list[1].shape[1]):
    dihedral_traj = psi_list[1][:,i]
    dihedral_traj_deg = (dihedral_traj * 180) / math.pi
    index = np.linspace(1,len(dihedral_traj), len(dihedral_traj)).astype('int')
    # Save torsion to output
    np.savetxt('1_DIHEDRALS/%s/OUTPUT/PSI/raw_data_psi_%d.dat' % (system_list[input_system],(i+1)),dihedral_traj_deg)
    # Histogram
    (n, bins) = np.histogram(dihedral_traj_deg, bins=number_of_bins, range=(-180.00, 180.00), normed=True)
    bincentre = 0.5*(bins[1:]+bins[:-1])
    index = np.linspace(1, len(bincentre), num = len(bincentre), dtype = int)
    # Total amount to be split over empty bins only
    total_bin_addition = 0.001
    all_bins = len(bincentre)
    non_zero = np.count_nonzero(n)
    zero_bins = all_bins - non_zero
    # Adds the bin_addition amount into all zero-count bins
    if zero_bins != 0:
        bin_addition = total_bin_addition/float(zero_bins)
        # Adds the bin_addition amount into all zero-count bins
        for j in xrange(len(n)):
            if n[j] == 0.0:
                n[j] = bin_addition
    #normalise
    n = n / (sum(n))
    data = np.vstack((index, n)).T
    np.savetxt('1_DIHEDRALS/%s/OUTPUT/PSI/psi_hist_%d.dat' % (system_list[input_system],(i+1)), data, fmt=['%d', '%.30f'])

---
#### Torsion number and atom index

In order to keep track of which torsional distribution relates to which atoms, a file is output with the torsion number and the atom indices for that torsion.

In [23]:
index = np.linspace(1,(psi_list[1].shape[1]),num=(psi_list[1].shape[1]))
index = index.reshape(len(index),1)
psi_atoms_array = np.array(psi_list[0])
psi_atoms_array =  np.hstack((index , psi_atoms_array))

In [24]:
np.savetxt('1_DIHEDRALS/%s/OUTPUT/PSI/1_psi_indices_list.dat' % system_list[input_system] , psi_atoms_array , fmt='%d')

#### Now repeat the same for Phi, Chi1 and Chi2
---
### PHI

In [20]:
for i in xrange(phi_list[1].shape[1]):
    dihedral_traj = phi_list[1][:,i]
    dihedral_traj_deg = (dihedral_traj * 180) / math.pi
    index = np.linspace(1,len(dihedral_traj), len(dihedral_traj)).astype('int')
    # Save torsion to output
    np.savetxt('1_DIHEDRALS/%s/OUTPUT/PHI/raw_data_phi_%d.dat' % (system_list[input_system],(i+1)),dihedral_traj_deg)
    # Histogram
    (n, bins) = np.histogram(dihedral_traj_deg, bins=number_of_bins, range = (-180.00, 180.00), normed=True)
    bincentre = 0.5*(bins[1:]+bins[:-1])
    index = np.linspace(1, len(bincentre), num = len(bincentre), dtype = int)
    total_bin_addition = 0.001
    all_bins = len(bincentre)
    non_zero = np.count_nonzero(n)
    zero_bins = all_bins - non_zero
    if zero_bins != 0:
        bin_addition = total_bin_addition/float(zero_bins)
        # Adds the bin_addition amount into all zero-count bins
        for j in xrange(len(n)):
            if n[j] == 0.0:
                n[j] = bin_addition
    n = n / (sum(n))
    data = np.vstack((index, n)).T
    np.savetxt('1_DIHEDRALS/%s/OUTPUT/PHI/phi_hist_%d.dat' % (system_list[input_system],(i+1)), data, fmt=['%d', '%.30f'])


index = np.linspace(1,(phi_list[1].shape[1]),num=(phi_list[1].shape[1]))
index = index.reshape(len(index),1)
phi_atoms_array = np.array(phi_list[0])
phi_atoms_array =  np.hstack((index , phi_atoms_array))
np.savetxt('1_DIHEDRALS/%s/OUTPUT/PHI/1_phi_indices_list.dat' % system_list[input_system] , phi_atoms_array , fmt='%d')

### CHI 1 

In [21]:
for i in xrange(chi1_list[1].shape[1]):
    dihedral_traj = chi1_list[1][:,i]
    dihedral_traj_deg = (dihedral_traj * 180) / math.pi
    index = np.linspace(1,len(dihedral_traj), len(dihedral_traj)).astype('int')
    # Save torsion to output
    np.savetxt('1_DIHEDRALS/%s/OUTPUT/CHI1/raw_data_chi1_%d.dat' % (system_list[input_system],(i+1)),dihedral_traj_deg)
    # Histogram
    (n, bins) = np.histogram(dihedral_traj_deg, bins=number_of_bins, range = (-180.00, 180.00), normed=True)
    bincentre = 0.5*(bins[1:]+bins[:-1])
    index = np.linspace(1, len(bincentre), num = len(bincentre), dtype = int)
    # Total amount to be split over empty bins only
    total_bin_addition = 0.001
    all_bins = len(bincentre)
    non_zero = np.count_nonzero(n)
    zero_bins = all_bins - non_zero
    # Adds the bin_addition amount into all zero-count bins
    if zero_bins != 0:
        bin_addition = total_bin_addition/float(zero_bins)
        # Adds the bin_addition amount into all zero-count bins
        for j in xrange(len(n)):
            if n[j] == 0.0:
                n[j] = bin_addition
    #normalise
    n = n / (sum(n))
    data = np.vstack((index, n)).T
    np.savetxt('1_DIHEDRALS/%s/OUTPUT/CHI1/chi1_hist_%d.dat' % (system_list[input_system],(i+1)), data, fmt=['%d', '%.30f'])
    
index = np.linspace(1,(chi1_list[1].shape[1]),num=(chi1_list[1].shape[1]))
index = index.reshape(len(index),1)
chi1_atoms_array = np.array(chi1_list[0])
chi1_atoms_array =  np.hstack((index , chi1_atoms_array))
np.savetxt('1_DIHEDRALS/%s/OUTPUT/CHI1/1_chi1_indices_list.dat' % system_list[input_system] , chi1_atoms_array , fmt='%d')

### CHI 2

In [22]:
for i in xrange(chi2_list[1].shape[1]):
    dihedral_traj = chi2_list[1][:,i]
    dihedral_traj_deg = (dihedral_traj * 180) / math.pi
    index = np.linspace(1,len(dihedral_traj), len(dihedral_traj)).astype('int')
    # Save torsion to output
    np.savetxt('1_DIHEDRALS/%s/OUTPUT/CHI2/raw_data_chi2_%d.dat' % (system_list[input_system],(i+1)),dihedral_traj_deg)
    # Histogram
    (n, bins) = np.histogram(dihedral_traj_deg, bins=number_of_bins, range = (-180.00, 180.00), normed=True)
    bincentre = 0.5*(bins[1:]+bins[:-1])
    index = np.linspace(1, len(bincentre), num = len(bincentre), dtype = int)
    # Total amount to be split over empty bins only
    total_bin_addition = 0.001
    all_bins = len(bincentre)
    non_zero = np.count_nonzero(n)
    zero_bins = all_bins - non_zero
    # Adds the bin_addition amount into all zero-count bins
    if zero_bins != 0:
        bin_addition = total_bin_addition/float(zero_bins)
        # Adds the bin_addition amount into all zero-count bins
        for j in xrange(len(n)):
            if n[j] == 0.0:
                n[j] = bin_addition
    #normalise
    n = n / (sum(n))
    data = np.vstack((index, n)).T
    np.savetxt('1_DIHEDRALS/%s/OUTPUT/CHI2/chi2_hist_%d.dat' % (system_list[input_system],(i+1)), data, fmt=['%d', '%.30f'])
    
index = np.linspace(1,(chi2_list[1].shape[1]),num=(chi2_list[1].shape[1]))
index = index.reshape(len(index),1)
chi2_atoms_array = np.array(chi2_list[0])
chi2_atoms_array =  np.hstack((index , chi2_atoms_array))
np.savetxt('1_DIHEDRALS/%s/OUTPUT/CHI2/1_chi2_indices_list.dat'% system_list[input_system] , chi2_atoms_array , fmt='%d')

### 2.8. If the first system has now run: Restart to compute second system
----------

#### To restart the session after running 0_system_A, use the drop down menu at the top: 

  --> `Kernel` --> `Restart and clear output`

Then change `input_system = 1` in **section 2.1** and rerun all cells up to here, for the second system. 

Only restart after running `0_system_A`. Do not restart once you have run `1_system_B`. 

----------

### 2.9. If you have now run both system A and B: move on to Step 3 of the workflow below 

-----------

-----------


### <span style="color:LightSlateGray">*To run as a script instead of in the notebook:* </span>
 <span style="color:LightSlateGray"> To run this for several systems, use the bash script `Scripts/1_dihedral_many_systems.sh` with `Scripts/1_dihedral.py`. 
First alter `1_dihedral.py`: system_list, filename_list, topology_filename and md_data.
Then run `1_dihedral_many_systems.sh` with 2 arguments: The number of systems and the number of bins to histogram the data. </span>

-----------

-----------




# <span style="color:teal"> Step 3 of the workflow: </span>
## <span style="color:teal"> Calculating KL between two systems </span>
---

---


A bash script will now loop thorugh all of the output files for the two systems which we created with 1_Dihedral.py and calculate the KL value. 

We run this script with several arguments. In the next cell, we print `number_angles_array` which gives how many psi, phi, chi1 and chi2 there are in the system. There is also a file saved in the folder 1_DIHEDRALS/**number_angles.dat** with this information.

In the cells below we can run this directly from the notebook. 

### <span style="color:LightSlateGray">*To run as a script instead of in the notebook:* </span>

` $ bash 2_run_all_KL.sh systemA_folder_name systemB_folder_name number_psi_angles number_phi_angles number_chi1_angles number_chi2_angles`

e.g. `bash ../Scripts/2_run_all_KL.sh 0_system_A 1_system_B 285 285 252 204` 

### 3.1. Print the number of angles for each torsion
---

In [ ]:
print (number_angles_array)


### 3.2. Run the bash script to calculate the KL between the two systems 
---
First, change directory to 1_DIHEDRALS as all output will be processed and saved in this folder.

#### Move to the folder `1_DIHEDRALS` :

In [ ]:
cd 1_DIHEDRALS

In [ ]:
# This may take a few minutes!
!bash ../Scripts/2_run_all_KL.sh 0_system_A 1_system_B 285 285 252 204

### 3.3.  Collect KL values and assign them to the correct residue
---

For each of the calculations for Psi, Phi, Chi1 and Chi2, we saved a file with the C$\alpha$ index and the torsion number, to later assign each to the correct residue. 

To do this, the script `Scripts/5_combining_psi_phi_chi.py` is used, and can be run from the cells below. This should be run from the folder `1_DIHEDRALS`. 

This will output a file with the number of the residue in the first column, the number of the C$\alpha$ atom in the second column, and the remaining 4 columns are the KL value for the Psi, Phi, Chi1 and Chi2 torsions respectively. 

From 2.6, we should already have the file
`DIHEDRALS_CA_angles_indexes.dat`, which looks like this:

| CA index | PSI | PHI | CHI1 | CHI2 |
| :--- | :-: | :-: | :--: | :--: |
| 8 | 0| 0 | 0 | 0| 
| 32 | 1 | 1 | 1 | 1| 
| 54 | 2 | 2 | 2 | 2| 
| 76 | 3 | 3 | 3 | 3| 

<br><br>

And then running `Scripts/5_combining_psi_phi_chi.py` will output 
`KL_residue_caindex_psi_phi_chi_chi.dat`, which looks like this: 

|Residue number | CA index | PSI KL | PHI KL | CHI1 KL | CHI2 KL |
| :--- | :-: | :-: | :--: | :--: |:--: |
| 1 | 8 | 0.013591 | 0.004713 | 0.014678 | 0.036920
| 2 | 32 | 0.011217 | 0.030264 | 0.022233 | 0.012993
| 3 | 54 | 0.077490 | 0.010198 | 0.067854 | 0.056849
| 4 | 76 | 0.023213 | 0.007356 | 0.198231 | 0.125124

<br><br>

**This script also outputs several other files:**

Each is a list of the KL of that torsion, for each residue. 

* KL_psi.dat

* KL_phi.dat

* KL_chi1.dat

* KL_chi2.dat


Also a file which sums the KL for *Phi+Psi* as "backbone"; and *Chi1+Chi2* as "sidechain".

* KL_backbone.dat

* KL_sidechain.dat


### <span style="color:LightSlateGray">*To run as a script instead of in the notebook:* </span>

`python script.py KL-output-directory`

<span style="color:LightSlateGray"> e.g.</span>

`python 5_combining_psi_phi_chi.py KL_OUTPUT/0_system_A_1_system_B`

---

As mentioned before, it is sensible to check that the output from the two systems is the same (to check for any errors with different topology, for example). 

`$ vimdiff 1_DIHEDRALS/0_system_A/DIHEDRALS_CA_angles_indexes.dat 1_DIHEDRALS/1_system_B/DIHEDRALS_CA_angles_indexes.dat`

If they are identical, copy one of these to the folder 1_DIHEDRALS:

In [ ]:
cp 0_system_A/DIHEDRALS_CA_angles_indexes.dat .




------

The KL calculation will have output values in a folder named: `1_DIHEDRALS/KL_OUTPUT/0_system_A_1_system_B`

To combine all the KL values, run the script in the cell below, `5_combining_psi_phi_chi.py`, with one argument which is the folder containing the KL output. 

---


In [ ]:
!python ../Scripts/5_combining_psi_phi_chi.py KL_OUTPUT/0_system_A_1_system_B


So now we have a file with the residue number, the CA index and the corresponding KL values for each torsion; and other files where we group together the KL for the backbone torsions and the sidechain torsions separately. 

------

------


# <span style="color:teal"> Step 4 of the workflow: </span>
## <span style="color:teal"> Visualising the output of the KL calculation </span>

---

---



Normally we visualise the KL caldulation by inputting these backbone and sidechain KL values into the b-factor column of a pdb, which can then be colour coded in pymol to show the high-KL regions. 

Plotting the KL values of sidechain and backbone against the residue number (below), shows that the higher values for the backbone and sidechain tend to be for different residues. So it is useful to plot in pymol the sidechain and backbone separately. 

---



In [ ]:
backbone = np.loadtxt("KL_backbone.dat")
sidechain = np.loadtxt("KL_sidechain.dat")

plt.plot(backbone,label="backbone")
plt.plot(sidechain,label="sidechain")
plt.xlabel("Residue number")
plt.ylabel("KL divergence")
plt.legend()


### 4.1 Loading the KL values into a pymol session
---

Open a terminal window and go to the folder `1_DIHEDRALS`

From here, run the pymol script `../Scripts/6_pymol_all_dihed.pml`


<img src="z_images/running_pymol_session.png" width="800">



This should launch a pymol session, loading the pdb structure from one system, and creating a few separate objects: One with the KL for the backbone torsions and another for the KL for the sidechain. The Psi, Phi, Chi1 and Chi2.

These are colour scaled individually: Since high KL values for each may not be similar size it can be useful to look at them separately. If you want to scale the colour plot to the same values for both backbone and sidechain, this can be done by altering pymol_all_dihed.pml (there is a section commented out that can be changed on line 30). 



A pymol session is then saved as **"Dihedral_KL.pse"**.

The output should look similar to below. Highest KL regions are red, and lowest are white. On the right, you can switch between backbone, sidechain, or individual torsions. 

<img src="z_images/pymol_session_Dihedrals.png">